# Marketing and Finance Reconciliation Dashboard

This notebook creates an interactive dashboard using Plotly Dash to reconcile marketing and finance data. The dashboard includes:
- Side-by-side ROAS comparison
- Customer Acquisition Cost (CAC) by channel
- Interactive funnel waterfall chart
- Revenue variance scatter plot
- A filterable table of misreporting anomalies
- Executive summary metrics

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output

# --- 1. Load and Prepare Data ---
spend_df = pd.read_csv('../data/marketing_spend.csv')
events_df = pd.read_csv('../data/funnel_events.csv')
revenue_mkt_df = pd.read_csv('../data/revenue_marketing.csv')
revenue_fin_df = pd.read_csv('../data/revenue_finance.csv')
anomalies_df = pd.read_csv('../data/anomalies.csv')

# Convert to datetime
spend_df['date'] = pd.to_datetime(spend_df['date'])
revenue_mkt_df['date'] = pd.to_datetime(revenue_mkt_df['date'])
revenue_fin_df['date'] = pd.to_datetime(revenue_fin_df['date'])
events_df['date'] = pd.to_datetime(events_df['timestamp']).dt.date
events_df['date'] = pd.to_datetime(events_df['date'])

# --- 2. Calculations for Dashboard ---

# ROAS Calculation
spend_by_channel = spend_df.groupby('campaign')['spend'].sum()
mkt_revenue_by_channel = revenue_mkt_df.groupby('campaign')['revenue'].sum()
fin_revenue_by_channel = revenue_mkt_df.merge(revenue_fin_df, on='date', suffixes=('_mkt', '_fin')).groupby('campaign_mkt')['revenue_fin'].sum()

roas_mkt = (mkt_revenue_by_channel / spend_by_channel).reset_index(name='roas')
roas_fin = (fin_revenue_by_channel / spend_by_channel).reset_index(name='roas')

# CAC Calculation (Assumption: a user checkout event counts as one acquisition)
daily_campaigns = spend_df[['date', 'campaign']].drop_duplicates()
events_with_campaign = pd.merge(events_df, daily_campaigns, on='date')
acquisitions = events_with_campaign[events_with_campaign['event_type'] == 'checkout'].groupby('campaign')['user_id'].nunique().reset_index(name='customers')
cac_df = pd.merge(acquisitions, spend_by_channel.reset_index(), on='campaign')
cac_df['cac'] = cac_df['spend'] / cac_df['customers']

# Funnel Data by Channel
funnel_steps = events_with_campaign.groupby(['campaign', 'event_type'])['user_id'].nunique().unstack(fill_value=0)
funnel_steps = funnel_steps[['page_view', 'add_to_cart', 'checkout']] # Ensure order

# Revenue Variance Data
mkt_rev_daily = revenue_mkt_df.groupby('date')['revenue'].sum()
fin_rev_daily = revenue_fin_df.groupby('date')['revenue'].sum()
variance_df = pd.concat([mkt_rev_daily, fin_rev_daily], axis=1).fillna(0)
variance_df.columns = ['mkt_revenue', 'fin_revenue']
variance_df['variance'] = variance_df['mkt_revenue'] - variance_df['fin_revenue']

# Executive Summary Metrics
total_spend = spend_df['spend'].sum()
total_mkt_revenue = revenue_mkt_df['revenue'].sum()
total_fin_revenue = revenue_fin_df['revenue'].sum()
overall_discrepancy = (total_mkt_revenue - total_fin_revenue) / total_fin_revenue

# --- 3. Build Dash App ---
app = JupyterDash(__name__)

app.layout = html.Div(style={'backgroundColor': '#f9f9f9'}, children=[
    html.H1('Marketing vs. Finance Reconciliation', style={'textAlign': 'center', 'padding': '20px'}),
    
    # Executive Summary
    html.Div([
        html.Div([html.H3(f'${total_spend:,.0f}'), html.P('Total Spend')], style={'textAlign': 'center', 'margin': '10px', 'padding': '10px', 'border': '1px solid #ddd', 'borderRadius': '5px'}),
        html.Div([html.H3(f'${total_fin_revenue:,.0f}'), html.P('Total Finance Revenue')], style={'textAlign': 'center', 'margin': '10px', 'padding': '10px', 'border': '1px solid #ddd', 'borderRadius': '5px'}),
        html.Div([html.H3(f'{total_fin_revenue/total_spend:.2f}x'), html.P('Overall Finance ROAS')], style={'textAlign': 'center', 'margin': '10px', 'padding': '10px', 'border': '1px solid #ddd', 'borderRadius': '5px'}),
        html.Div([html.H3(f'{overall_discrepancy:.1%}'), html.P('Revenue Discrepancy')], style={'textAlign': 'center', 'margin': '10px', 'padding': '10px', 'border': '1px solid #ddd', 'borderRadius': '5px'}),
    ], style={'display': 'flex', 'justifyContent': 'center'}),
    
    # Charts
    html.Div([
        dcc.Graph(id='roas-chart', figure=
            go.Figure([
                go.Bar(name='Marketing ROAS', x=roas_mkt['campaign'], y=roas_mkt['roas']),
                go.Bar(name='Finance ROAS', x=roas_fin['campaign'], y=roas_fin['roas'])
            ]).update_layout(title_text='Marketing vs Finance ROAS', barmode='group')
        , style={'width': '49%', 'display': 'inline-block'}),
        
        dcc.Graph(id='cac-chart', figure=
            px.bar(cac_df, x='campaign', y='cac', title='Customer Acquisition Cost (CAC) by Channel')
        , style={'width': '49%', 'display': 'inline-block'}),
    ]),

    html.Div([
       dcc.Graph(id='variance-scatter', figure=
            px.scatter(variance_df, x='fin_revenue', y='mkt_revenue', hover_data=[variance_df.index], title='Daily Revenue Variance (Marketing vs Finance)')
       , style={'width': '49%', 'display': 'inline-block'}),
        
       html.Div([
           html.H4('Funnel by Channel', style={'textAlign': 'center'}),
           dcc.Dropdown(
               id='channel-dropdown',
               options=[{'label': i, 'value': i} for i in funnel_steps.index],
               value=funnel_steps.index[0]
           ),
           dcc.Graph(id='funnel-chart')
       ], style={'width': '49%', 'display': 'inline-block'}),
    ]),

    # Misreporting Table
    html.H3('Misreporting Incidents', style={'padding': '20px', 'textAlign': 'center'}),
    dash_table.DataTable(
        id='anomaly-table',
        columns=[{'name': i, 'id': i} for i in anomalies_df.columns],
        data=anomalies_df.to_dict('records'),
        filter_action='native',
        sort_action='native',
        page_size=10,
        style_table={'overflowX': 'auto'}
    )
])

@app.callback(
    Output('funnel-chart', 'figure'),
    Input('channel-dropdown', 'value')
)
def update_funnel_chart(selected_channel):
    channel_funnel = funnel_steps.loc[selected_channel]
    fig = go.Figure(go.Funnel(
        y = channel_funnel.index,
        x = channel_funnel.values
    ))
    fig.update_layout(title_text=f'Funnel for {selected_channel}')
    return fig

app.run_server(mode='inline')